In [1]:
from os import path
import nexradaws
import time
import tempfile
import os
import shutil
import pyart
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from cartopy.feature import NaturalEarthFeature
import matplotlib.colors as colors


import numpy as np

import pytz
from copy import deepcopy

from math import atan2 as atan2
import warnings

%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def custom_plotfunc(ds, fig, tt, *args, **kwargs):
    projection = ccrs.PlateCarree()
    lat = 38.89838028
    lon = -106.94321442
    map_panel_axes = [0.05, 0.05, .4, .80]
    x_cut_panel_axes = [0.6, 0.10, .4, .25]
    y_cut_panel_axes = [0.6, 0.50, .4, .25]

    ax1 = fig.add_axes(map_panel_axes, projection=projection)
    
    ds.DBZ.sel(z=1000, method="nearest").isel(time=tt).plot.pcolormesh(vmin=-10, vmax=64,
                                              cmap=pyart.graph.cm_colorblind.HomeyerRainbow, ax=ax1,
                                                                               add_colorbar=False)
    
    gl = ax1.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
    ax1.add_feature(cfeature.COASTLINE)
    
    ax2 = fig.add_axes(x_cut_panel_axes)
    ds.DBZ.sel(x=lon, method="nearest").isel(time=tt).plot.pcolormesh(vmin=-10, vmax=64,
                                              cmap=pyart.graph.cm_colorblind.HomeyerRainbow, ax=ax2)
    
    y_axis = ax2.axes.get_yaxis()
    y_axis.set_label_text('Height (m)')
    
    ax2.set_ylim([0, 8000])
    

    ax3 = fig.add_axes(y_cut_panel_axes)
    ds.DBZ.sel(y=lat, method="nearest").isel(time=tt).plot.pcolormesh(vmin=-10, vmax=64,
                                              cmap=pyart.graph.cm_colorblind.HomeyerRainbow, ax=ax3)
    
    y_axis = ax3.axes.get_yaxis()
    y_axis.set_label_text('Height (m)')
    
    ax3.set_ylim([0, 8000])

    gl.xlabel_style = {'fontsize': 10}
    gl.ylabel_style = {'fontsize': 10}
    
    return None, None




In [3]:
data_dir = '/eagle/CPOL/SAIL/202112_glued/'
all_files = os.listdir(data_dir)

In [4]:
for i in range(len(all_files)):
    all_files[i] = os.path.join(data_dir, all_files[i])

In [ ]:
radar = pyart.io.read(all_files[400])
grids = pyart.map.grid_from_radars((radar),(17,601,601),
                       ((0.,8000.),(-30000.,30000.),(-30000.,30000.)), 
                                       weighting_function='Barnes2')

In [ ]:
xgrids = grids.to_xarray()
xgrids = xgrids.set_index(x='lon', y='lat')

In [ ]:
figme = plt.figure(figsize=[15,10])
custom_plotfunc(xgrids, figme, 0)

In [ ]:
gtime = xgrids.time.data[0]

In [ ]:
fname = gtime.strftime('/eagle/projects/CPOL/SAIL/grids/%Y%m/xprecipradar_guc_grid_%Y%m%d-%H%M%S.b1.nc')

In [ ]:
print(fname)

In [ ]:
xgrids.to_netcdf(fname)

In [5]:
def granule(filename):
    radar = pyart.io.read(filename)
    grids = pyart.map.grid_from_radars((radar),(17,601,601),
                       ((0.,8000.),(-30000.,30000.),(-30000.,30000.)), 
                                       weighting_function='Barnes2')
    xgrids = grids.to_xarray()
    xgrids = xgrids.set_index(x='lon', y='lat')
    gtime = xgrids.time.data[0]
    fname = gtime.strftime('/eagle/projects/CPOL/SAIL/grids/%Y%m/xprecipradar_guc_grid_%Y%m%d-%H%M%S.b1.nc')
    xgrids.to_netcdf(fname)
    return fname
    

In [6]:
granule(all_files[20])

'/eagle/projects/CPOL/SAIL/grids/202112/xprecipradar_guc_grid_20211215-181231.b1.nc'

In [7]:
from dask.distributed import Client, LocalCluster


In [8]:
from dask.distributed import Client, LocalCluster

#cluster = LocalCluster()



In [10]:
cluster = LocalCluster()

cluster.scale(16)  # Sets the number of workers to 10
cluster.adapt(minimum=8, maximum=16)
client = Client(cluster)
client
client

Client Scheduler: tcp://127.0.0.1:44800 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 128 Memory: 1.08 TB


In [25]:
future = client.map(granule, all_files[-500:-400])

In [26]:
my_data = client.gather(future)

In [ ]:
len(my_data)

In [ ]:
my_data

CPOL has no allocation on cooley.
Projects available: 
For assistance, contact support@alcf.anl.gov
Filter /usr/bin/ni-account.filter failed
